# Digital Twin - Constant product AMM


In [92]:
import copy
import json

def init_genesis():
    return  {
        "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.00},
        "Trader":{"r_1":100,"r_2":100,"s":0},
        "LP":{"r_1":0,"r_2":0,"s":100}
    }

def swapToAsset1(state,inputs):
    agent = inputs[0]
    dA2 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA1 = state["AMM"]["r_1"]/(state["AMM"]["r_2"]+dA2*feeFactor)*dA2*feeFactor
    if dA2>0 and state[agent]["r_2"]-dA2 >= 0 :
        state["AMM"]["r_2"]+=dA2
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]-=dA1
        state[agent]["r_1"]+=dA1

def swapToAsset2(state,inputs):
    agent = inputs[0]
    dA1 = inputs[1]
    feeFactor = (1-state["AMM"]["fee"])
    dA2 = state["AMM"]["r_2"]/(state["AMM"]["r_1"]+dA1*feeFactor)*dA1*feeFactor
    if dA1>0 and state[agent]["r_1"]-dA1 >= 0 :
        state["AMM"]["r_1"]+=dA1
        state[agent]["r_1"]-=dA1
        state["AMM"]["r_2"]-=dA2
        state[agent]["r_2"]+=dA2

def addLiquidity(state,inputs):
    agent = inputs[0]
    R1=state["AMM"]["r_1"]
    R2=state["AMM"]["r_2"]
    S= state["AMM"]["s"]
    dA1=min(inputs[1],R1/R2*inputs[2])
    dA2=min(inputs[2],R2/R1*inputs[1])
    if (dA1 <= state[agent]["r_1"] and dA2 <= state[agent]["r_2"]) and (dA1 > 0 and dA2 > 0):
        state[agent]["r_1"]-=dA1
        state[agent]["r_2"]-=dA2
        state["AMM"]["r_1"]+=dA1
        state["AMM"]["r_2"]+=dA2
        dS = min(dA1/R1, dA2/R2) * S
        state["AMM"]["s"] += dS
        state[agent]["s"]+=dS

def removeLiquidity(state,inputs):
    dS = inputs[1]
    agent = inputs[0]
    if dS > 0 and state[agent]["s"]-dS>=0 and state["AMM"]["s"]-dS>=0:
        DR = (1-dS/state["AMM"]["s"])
        R1=state["AMM"]["r_1"]
        R2=state["AMM"]["r_2"]
        state[agent]["s"]-=dS
        state["AMM"]["r_1"]=R1*DR
        state["AMM"]["r_2"]=R2*DR
        state[agent]["r_1"]+=R1-state["AMM"]["r_1"]
        state[agent]["r_2"]+=R2-state["AMM"]["r_2"]
        state["AMM"]["s"]-=dS

def pretty_print(obj):
    print(json.dumps(obj, sort_keys=True, indent=4))

def nice_print(self):
    if type(self)==float:
        return  round(self,3)
    if type(self)==int:
        return  self
    if type(self)==list:
        return  list(map(lambda l: print(l),self))
    if type(self)==dict:
        return  dict(map(lambda kv: (kv[0], print(kv[1])),self.items()))

def evolve(state, actionStack):
    history = [copy.deepcopy(state)]
    for action in actionStack:
        action[0](state,action[1])
        history.append(copy.deepcopy(state))
    return history 

### Testing the swapToAsset2 function

We create a genesis state with:

- the AMM having 100 Token 1, 100 Token 2, 100 pool shares, and a fee of 0.0
- the Trader having 100 Token 1, 100 Token 2, and no pool shares
- the Liquidity Provider has 0 Token 1, 0 Token 2, and 100 pool shares
Then we apply the swapToAsset2 function and look at the new genesis state by typing genesis

which will print the values of the variables to the screen. 

In [93]:
# test 1
state = init_genesis()
swapToAsset2(state,["Trader",10])
pretty_print(state)

{
    "AMM": {
        "fee": 0.0,
        "r_1": 110,
        "r_2": 90.9090909090909,
        "s": 100
    },
    "LP": {
        "r_1": 0,
        "r_2": 0,
        "s": 100
    },
    "Trader": {
        "r_1": 90,
        "r_2": 109.0909090909091,
        "s": 0
    }
}


Lets now take a look at a more elaborate state transition and perform two different swaps after one another: we append exactly the swap in opposite direction expecting that this restores the state.

In [94]:
state = init_genesis()

swapToAsset2(state,["Trader",10])
swapToAsset1(state,["Trader",9.0909090909091])

pretty_print(state)

{
    "AMM": {
        "fee": 0.0,
        "r_1": 99.99999999999999,
        "r_2": 100.0,
        "s": 100
    },
    "LP": {
        "r_1": 0,
        "r_2": 0,
        "s": 100
    },
    "Trader": {
        "r_1": 100.00000000000001,
        "r_2": 100.0,
        "s": 0
    }
}


Create an actionList that represents the following actions (in this order):

1. The trader puts 50 Asset 1 to the AMM, in exchange for Asset 2

2. The trader deposits 25 Asset 2 to the AMM, in exchange for Asset 1

How much of Asset 1 does the trader have at this point? Might this surprise the Trader? 

In [95]:
# test 3
state = init_genesis()
actionList = [
        [ swapToAsset1 ,  ["Trader", 50.0] ],
        [ swapToAsset2 ,  ["Trader", 25.0] ]
]
evolve(state, actionList)
#pretty_print(genesis)

# How much of Asset 1 does the trader have at this point? Might this surprise the Trader?  
state["Trader"]["r_1"]

108.33333333333331

### Try Another One

As the code is currently written, it would be possible for a careless user to input a genesis block that contained) invalid information (such as a negative value for the tokens). Write a function called check_genesis_block that returns True if all inputs to the genesis block are valid, and False if there are invalid inputs. 

You can check that your function works as intended by creating three examples that are invalid in some way, and being sure that your function 

In [96]:
def check_genesis_block(state):
    return state["AMM"]["r_1"] >= 0 and \
        state["AMM"]["r_2"] >= 0 and \
        state["AMM"]["s"] >= 0 and \
        state["AMM"]["fee"] >= 0 and \
        state["Trader"]["r_1"] >= 0 and \
        state["Trader"]["r_2"] >= 0 and \
        state["Trader"]["s"] >= 0 and \
        state["LP"]["r_1"] >= 0 and \
        state["LP"]["r_2"] >= 0 and \
        state["LP"]["s"] >= 0

state = init_genesis()
print(check_genesis_block(state))

True


### Updating Pool Fees

In this section, we have assumed that the fees for the liquidity pool remain the same as time goes on. Write a function called update_pool_fees that changes the fee charged by the AMM, so we may adjust that in our simulation.

In [97]:
def update_pool_fees(state, new_fee):
    state["AMM"]["fee"] = new_fee

state = init_genesis()
update_pool_fees(state, 0.3)
print(state)

{'AMM': {'r_1': 100, 'r_2': 100, 's': 100, 'fee': 0.3}, 'Trader': {'r_1': 100, 'r_2': 100, 's': 0}, 'LP': {'r_1': 0, 'r_2': 0, 's': 100}}


### Trading a Percentage of the Wallet

In this section, we have assumed that the user wanted to swap a particular amount of tokens. What if instead we want the user to be able to swap a certain percentage of their holdings?

 Write a function called swapToAsset2_pct that swaps a certain percentage of the user's Asset 1 funds in exchange for Asset 2 

(Hint: Write a helper function called pctToAmount that converts percent to amount, and then can call the swapToAsset2 function.

In [98]:
def pctToAmount(state, pct):
    if pct > 100:
        pct = 100
    elif pct < 0:
        pct = 0

    pct = pct / 100
    return state["Trader"]["r_1"]*pct

def swapToAsset2_pct(state, inputs):
    inputs[1] = pctToAmount(state, inputs[1])
    return swapToAsset2(state, inputs)

genesis = init_genesis()
swapToAsset2_pct(genesis, ["Trader", 3])
print(genesis)


{'AMM': {'r_1': 103.0, 'r_2': 97.0873786407767, 's': 100, 'fee': 0.0}, 'Trader': {'r_1': 97.0, 'r_2': 102.9126213592233, 's': 0}, 'LP': {'r_1': 0, 'r_2': 0, 's': 100}}


### Pretty Print in Fiat Values

In this section, we have been completely agnostic as to the underlying value that a user's token holding may have in terms of a reference currency such as USD or EURO. Write a function fiat_print_holdings that takes in the names and prices of the tokens as a dictionary, then gives the value of the user's portfolio in the reference currency.

In [99]:
def fiat_print_holdings(state, agent, token_1, token_2, ref_currency):
    token_1_amount = state[agent][token_1["name"]]*token_1["price"]
    token_2_amount = state[agent][token_2["name"]]*token_2["price"]
    return  token_1_amount + token_2_amount

state = init_genesis()
token_1 = {"name": "r_1", "price": 5}
token_2 = {"name":"r_2", "price": 3}
fiat_print_holdings(state, "Trader", token_1, token_2 , "USD") 

800

Since the trader has holdings equivalent to 800 USD. 
This allows us to simulate impermanent loss by considering the fact that the value of the assets the trader holds may change.

### Verifying the Digital Twin

To trust the results of the digital twin, we need to be completely sure that it captures the behavior of the system we designed. 

Looking at our finished digital Twin, we want to be sure: does it really behave as it is meant to? This needs to be verified. To do so, we create simple test scenarios through which our simulations can run. These scenarios should be simple enough that we can verify them by hand. Let us recall what we expect from a well-coded digital Twin according to the business requirements:

**During a swap action,**
- the invariant shall remain unchanged (assuming no fees)
- the total number of tokens of Asset 1 in the system shall be unchanged
- the total number of tokens of Asset 2 in the system shall be unchanged
- given a specific amount of one Asset the correct amount of the other Asset shall be returned.

**During a liquidity removal**
- the total number of tokens of Asset 1 shall be unchanged,
- the total number of tokens of Asset 2 shall be unchanged,
- the correct number of shares shall be taken and burned,
- the correct amount of each Asset shall be transferred.

**During a liquidity insertion**
- the total number of tokens of Asset 1 shall be unchanged,
- the total number of tokens of Asset 2 shall be unchanged,
- the correct number of shares shall be minted and distributed,
the correct amount of each Asset shall be transferred.

For each of these requirements we need to think about good test scenarios that our digital Twin can simulate. Looking at the list above we see that the some of the actions have requirements in common.

Checking that the invariant is preserved during swap actions requires a calculation of the invariant for a given state. 
The conservation of tokens can be easily checked by counting them. 

Let's define three helpful functions:
- invariant
- asset1
- asset2

In [100]:
# it returns k, the invariant of the AMM 
def invariant(state):
    return state["AMM"]["r_1"] * state["AMM"]["r_2"]

# how many asset1 do we have in the state
def asset1(state):
    return state["AMM"]["r_1"]+state["Trader"]["r_1"]+state["LP"]["r_1"]

# how many asset2 do we have in the state
def asset2(state):
    return state["AMM"]["r_2"]+state["Trader"]["r_2"]+state["LP"]["r_2"]

First, let's use the invariant function to see if things work as intended. 

In [101]:
genesis = init_genesis()

state = copy.deepcopy(genesis)

swapToAsset2(state,["Trader",13])

invariant(genesis)==invariant(state)

True

### Verifying the System (and the Test) 

That is, we just create a genesis state which is then copied into the variable state that evolves through the swap action. In the last line we just check whether or not the invariance is identical for genesis state and evolved state. As can be expected, this test returns True. The same test can be performed for a swap in opposite direction. 

Keep in mind that the invariant is only preserved as long as there are no fees. Knowing scenarios in which the tests fail is important to verify also that the test in fact can fail: whenever possible, test the test!

You Try It: Verify that the test for preserved liquidity fails if nonzero fees are set.

In [102]:
genesis = init_genesis()

state = copy.deepcopy(genesis)

update_pool_fees(state, 0.3)

swapToAsset2(state,["Trader",13])

invariant(genesis)==invariant(state)

False

Now let's look at conservation of number of tokens in the system. 

In [103]:
state = init_genesis()

swapToAsset1(state,["Trader",13])

asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

True

Testing Individually or In Tandem

In the code block above, we combined the test for the asset1 and asset2 function. 

If we repeat the test for every state transition function we have coded, we can verify token conservation individually. We can also make sure this test will fail when appropriate by providing manipulated states:

In [104]:
genesis = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":100,"r_2":100,"s":0},
    "Liquidator":{"r_1":0,"r_2":0,"s":100}
    }

state = {
    "AMM":{"r_1":100,"r_2":100,"s":100,"fee":0.0},
    "Trader":{"r_1":0,"r_2":100,"s":0},
    "Liquidator":{"r_1":0,"r_2":0,"s":100}
    }
    
asset1(genesis)==asset1(state) and asset2(genesis)==asset2(state)

KeyError: 'LP'